## 201810824 현소미

## 1-1답 geth@8446 admin.nodeInfo 출력

In [56]:
!geth --exec "admin.nodeInfo.protocols" attach http://localhost:8446

{
  eth: {
    config: {
      chainId: 36,
      eip150Hash: "0x0000000000000000000000000000000000000000000000000000000000000000",
      eip155Block: 0,
      eip158Block: 0,
      homesteadBlock: 0
    },
    difficulty: 87981407,
    genesis: "0x5704d029fe80f4fb605c0cb5e31d591511f10a46a0cb8166f97d8d559f9bc5b0",
    head: "0x4395434813856174f0c05569e5a3e49617cdf2859e5bee6948187d3c520d4f1f",
    network: 36
  }
}


In [57]:
%%writefile midterm1-1.js
console.log('1-2 accounts: ', eth.accounts);
console.log('1-2 blocknumber: ', eth.blockNumber);
var bal=eth.getBalance(eth.accounts[0]);
console.log('1-2 balance in ether: ', web3.fromWei(bal, "ether"));
var bal1=eth.getBalance(eth.coinbase);
var bal2_before=eth.getBalance(eth.accounts[1]);
console.log('sender balance in ether: ', web3.fromWei(bal1,"ether"));
console.log('receiver balance in ether: ', web3.fromWei(bal2_before,"ether"));
var t=eth.sendTransaction({from:eth.accounts[0], to:eth.accounts[1],value:web3.toWei(1.1,"ether")});
console.log('1-3 transactionhash: ', t);
console.log('...mining start');
miner.start(1);admin.sleepBlocks(1);miner.stop();
console.log('mining done...');
console.log("1-4 transaction receipt :",JSON.stringify(eth.getTransactionReceipt(t)));
var bal2_after=eth.getBalance(eth.accounts[1]);
console.log('1-5 :', bal2_after);
console.log('1-5balance difference: ',bal2_after-bal2_before);
console.log('1-5blocknumber: ', eth.blockNumber);

Overwriting midterm1-1.js


## 1-2,1-3,1-4,1-5답 

In [58]:
!geth --exec "loadScript('midterm1-1.js')" attach http://localhost:8446

1-2 accounts:  0x54e81eae4e6a73a977ee3d56e75df4e02b059455,0x85fcab4af316d42bb1e5e03f7ef7ccd1c04595f7
1-2 blocknumber:  627
1-2 balance in ether:  3121.8
sender balance in ether:  3121.8
receiver balance in ether:  13.2
1-3 transactionhash:  0x9e344c55532a87504e95d979d8af88d461928c0b0b4871b515495bc09ff53da3
...mining start
mining done...
1-4 transaction receipt : {"blockHash":"0xaa6e40577ce6ad81c2db2d3e885209c4df4ac6c6cb201ea9abd06a67b5b0c508","blockNumber":628,"contractAddress":null,"cumulativeGasUsed":21000,"from":"0x54e81eae4e6a73a977ee3d56e75df4e02b059455","gasUsed":21000,"logs":[],"logsBloom":"0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

## 2-1 답

In [1]:
%%writefile src/Math.sol
pragma solidity ^0.4.25;
contract Math {
    uint256 a=1;
   function powerOf2(uint input) public view returns (uint256){
       for(uint i=0;i<input;i++){
           a*=2;
       }
       return a;
   }
}

Overwriting src/Math.sol


## 2-2 답 abi, gas, bin 출력

In [2]:
!solc --abi src/Math.sol


======= src/Math.sol:Math =======
Contract JSON ABI 
[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}]


src/Math.sol:6:12: Warning: Function declared as view, but this expression (potentially) modifies the state and thus requires non-payable (the default) or payable.
           a*=2;

           ^


In [3]:
!solc --bin src/Math.sol


======= src/Math.sol:Math =======
Binary: 
6080604052600160005534801561001557600080fd5b5060e4806100246000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063bdacc0cf146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b600080600090505b8281101560ad57600260008082825402925050819055508080600101915050608a565b6000549150509190505600a165627a7a7230582054d37092c3002ecdf61d3dc4aa9b7965d66636a800fe51c8aa9887266fcf78b00029


src/Math.sol:6:12: Warning: Function declared as view, but this expression (potentially) modifies the state and thus requires non-payable (the default) or payable.
           a*=2;

           ^


In [4]:
!solc --gas src/Math.sol


======= src/Math.sol:Math =======
Gas estimation:
construction:
   20105 + 45600 = 65705
external:
   powerOf2(uint256):	infinite


src/Math.sol:6:12: Warning: Function declared as view, but this expression (potentially) modifies the state and thus requires non-payable (the default) or payable.
           a*=2;

           ^


## 답2-3 배포 스크립트 작성

In [5]:
%%writefile src/mathDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8446"));
}
var _abiArray=[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}];
var _bin="6080604052600160005534801561001557600080fd5b5060e4806100246000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063bdacc0cf146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b600080600090505b8281101560ad57600260008082825402925050819055508080600101915050608a565b6000549150509190505600a165627a7a7230582054d37092c3002ecdf61d3dc4aa9b7965d66636a800fe51c8aa9887266fcf78b00029";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x54e81eae4e6a73a977ee3d56e75df4e02b059455", gas: 3000000, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/mathDeploy.js


In [6]:
!node src/mathDeploy.js

0xa677E96360c770a9e61e82B9a652BD3d15d23604


## 답 2-4 사용 스크립트 작성

In [22]:
%%writefile src/mathUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8446"));
var _abiArray=[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}];
var Math = new web3.eth.Contract(_abiArray,"0xa677E96360c770a9e61e82B9a652BD3d15d23604");
Math.methods.powerOf2(8).send({from:"0x54e81eae4e6a73a977ee3d56e75df4e02b059455",gas:670000});
Math.methods.powerOf2(8).call().then(function(uint) {console.log(uint);});
Math.methods.powerOf2(32).send({from:"0x54e81eae4e6a73a977ee3d56e75df4e02b059455",gas:670000});
Math.methods.powerOf2(32).call().then(function(uint) {console.log(uint);});

Overwriting src/mathUse.js


## 답 2-4 각각 8, 32 입력결과 출력

In [8]:
!node src/mathUse.js

256
4294967296


## 2-5 못풀었습니다.

In [36]:
%%writefile scripts/powerOf2.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@0.20.5/dist/web3.min.js"></script>
<script type="text/javascript">
    //var Web3 = require('web3');  //can be removed
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8446"));

    function displaymath() {
        //web3@0.20.x
        var smathContract = web3.eth.contract([{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}]);
        var Math = smathContract.at("0xa677E96360c770a9e61e82B9a652BD3d15d23604");
        var Math=Math.powerOf2.call(-32);
        document.getElementById('powerOf2').innerText = "The message from blockchain: " + Math;
        document.getElementById('web3ver').innerText = 'web3 version 0.20.x';
        document.getElementById('web3coinbase').innerText = 'coinbase: ' + web3.eth.coinbase;

        //web3@1.2.x - can not instanciate a contract
        //alert("1.2.5");
        //var shelloContract = new web3.eth.Contract([{"constant":true,"inputs":[],"name":"sayHello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"}],"0x28a63917122075B10BB89475CC0d519E041BE3FB");
        //shelloContract.methods.sayHello().call().then(function(str) {
        //    alert(str);  //test
        //    document.getElementById('sayhello').innerText = "The message from blockchain: " + str;
        //});
    }
</script>
</head>
<body>
    <h1>MATH</h1>
    <button type="button" onClick="displaymath();">Display Math</button>
    <div></div>
    <div id="powerOf2"></div>
    <div id="web3ver"></div>
    <div id="web3coinbase"></div>
</body>
</html>

Overwriting scripts/powerOf2.html
